In [129]:
import pandas as pd
import numpy as np

# task_1
df = pd.read_csv("horse_data.csv")
columns = [
    "surgery?",
    "Age",
    "rectal temperature",
    "pulse",
    "respiratory rate",
    "temperature of extremities",
    "pain",
    "outcome",
]
df = df.iloc[:, [0, 1, 3, 4, 5, 6, 10, 22]]

df.columns = columns
df.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,1,39.2,88,20,?,3,3
1,2,1,38.30,40,24,1,3,1
2,1,9,39.10,164,84,4,2,2
3,2,1,37.30,104,35,?,?,2
4,2,1,?,?,?,2,2,1


In [130]:
# task_2
df.value_counts()
df = df.map(lambda x: pd.to_numeric(x, errors="coerce"))
df['Age'] = df['Age'].replace({9: 2})

def validate_values(row, *args):
    if row in args:
        return row
    else:
        return None

df["surgery?"] = df["surgery?"].apply(validate_values, args=(1, 2))
df["Age"] = df["Age"].apply(validate_values, args=(1, 2))
df["temperature of extremities"] = df["temperature of extremities"].apply(validate_values, args=(1, 2, 3, 4))
df["pain"] = df["pain"].apply(validate_values, args=(1, 2, 3, 4, 5))
df["outcome"] = df["outcome"].apply(validate_values, args=(1, 2, 3))

def iqr_func(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return series[(series >= lower_bound) & (series <= upper_bound)]

print('AS IS: '+str(df['rectal temperature'].mean().round(2)))
df['rectal temperature'] = iqr_func(df['rectal temperature'])
print('Manual: '+str(df[(df['rectal temperature'] > 30) & (df['rectal temperature'] < 50)]['rectal temperature'].mean().round(2)))
print('IQR_mean: '+str(df['rectal temperature'].mean().round(2)))
print('IQR_median: '+str(df['rectal temperature'].median().round(2)))
print('---')
print('AS IS: '+str(df['pulse'].mean().round(2)))
df['pulse'] = iqr_func(df['pulse'])
print('Manual: '+str(df[(df['pulse'] > 10) & (df['pulse'] < 150)]['pulse'].mean().round(2)))
print('IQR_mean: '+str(df['pulse'].mean().round(2)))
print('IQR_median: '+str(df['pulse'].median().round(2)))
print('---')
print('AS IS: '+str(df['respiratory rate'].mean().round(2)))
df['respiratory rate'] = iqr_func(df['respiratory rate'])
print('Manual: '+str(df[(df['respiratory rate'] > 7) & (df['respiratory rate'] < 11)]['respiratory rate'].mean().round(2)))
print('IQR_mean: '+str(df['respiratory rate'].mean().round(2)))
print('IQR_median: '+str(df['respiratory rate'].median().round(2)))

AS IS: 38.17
Manual: 38.17
IQR_mean: 38.17
IQR_median: 38.2
---
AS IS: 71.93
Manual: 70.27
IQR_mean: 70.27
IQR_median: 64.0
---
AS IS: 30.43
Manual: 9.33
IQR_mean: 26.71
IQR_median: 24.0


In [131]:
(df.isna().mean() * 100).round(2)

surgery?                       0.33
Age                            0.00
rectal temperature            24.75
pulse                          9.70
respiratory rate              25.08
temperature of extremities    18.73
pain                          18.39
outcome                        0.33
dtype: float64

In [132]:
# task_3
df = df.dropna(subset=['surgery?']) # Строки с пропущенными значениями удаляются так как это критически важный показатель для использования остальных параметров, 
                                    # его надо знать точно
                                    
df['rectal temperature'] = df['rectal temperature'].fillna(df['rectal temperature'].mean())  # данный параметр, судя по описанию, имеет явное нормальное распределение,
                                                                                             # и все методы расчета считают его практически идентично, поэтому заполним средним значением
                                                                                             
df = df.dropna(subset=['pulse']) # Несмотря на линейность распределения, данный параметр сильно отличается для нормального состояния разных пород лошадей и так как у нас отсутствует
                                 # категоризация пород и пропуски около 10% то лучше удалить
                                 
df = df.drop(columns=['respiratory rate']) # удаляем весь столбец тк, большие флуктуации данных сильно отличающихся от нормы, к тому же данный показатель не cильно важен, как сказано в                                           # документации

df = df.dropna(subset=['temperature of extremities'])
df = df.dropna(subset=['pain']) # эти 2 параметра как сказано в документации не должны рассматриваться как некие упорядоченные или дискретные величины, поэтому лучше удалить

df = df.dropna(subset=['outcome']) # Строки с пропущенными значениями удаляются так как это критически важный показатель для использования остальных параметров, 
                                    # его надо знать точно

(df.isna().mean() * 100).round(2)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 203 entries, 1 to 297
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    203 non-null    float64
 1   Age                         203 non-null    int64  
 2   rectal temperature          203 non-null    float64
 3   pulse                       203 non-null    float64
 4   temperature of extremities  203 non-null    float64
 5   pain                        203 non-null    float64
 6   outcome                     203 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 12.7 KB
